In [2]:
from ase.io import read,write
import numpy as np
atoms=read('gulp.xyz',format='xyz')
atoms.positions
atoms.set_cell([[21.889317,0,0], [0,21.889317,0],[0,0,21.889317]])

In [3]:
write('optimized.vasp',atoms,format='vasp',vasp5=True)

In [35]:
#check nearest distance
natom=len(atoms.positions)
dist=np.zeros((natom,natom,3))
from tqdm import tqdm
from nearest import find_nearest
for i in tqdm(range(natom)):  
    for j in range(i):
        xdc,ydc,zdc,rmin=find_nearest(atoms,i,j)
        dist[i,j]=np.array([xdc,ydc,zdc])

100%|██████████| 512/512 [01:07<00:00,  7.60it/s]


In [5]:
GULP_distx=np.loadtxt('distx.dat')[:,2]
GULP_disty=np.loadtxt('disty.dat')[:,2]
GULP_distz=np.loadtxt('distz.dat')[:,2]

In [6]:
GULP_distx

array([10.14206505,  7.88180726, -2.26025778, ..., -9.25122951,
       -7.09069481, -2.65821372])

In [7]:
from tqdm import tqdm
counter=0
diff=[]
for i in tqdm(range(natom)):  
    for j in range(i):
        diffx=np.abs(dist[i,j,0]-GULP_distx[counter])
        diffy=np.abs(dist[i,j,1]-GULP_disty[counter])
        diffz=np.abs(dist[i,j,1]-GULP_disty[counter])
        diff.append([diffx,diffy,diffz])
        #print(diffx,diffy,diffz)
        counter+=1

100%|██████████| 512/512 [00:00<00:00, 584.80it/s]


In [8]:
np.max(np.array(diff))

9.980016812960457e-10

距離の計算は間違ってないように見える

In [10]:
atoms.get_masses()

array([28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
       28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085,
      

In [11]:
structure_file='optimized.vasp'
FC_file='FORCE_CONSTANTS_2ND'

from thermal_conductivity_AF import get_Vij, get_Sij
Vx, Vy, Vz=get_Vij(structure_file,FC_file)

In [12]:
import numpy as np 
natom=len(atoms.positions)
eigenvector=np.loadtxt('eigr.dat').reshape((natom*3,natom*3))
omega=np.loadtxt('freq.dat')

In [13]:
Sx,Sy,Sz=get_Sij(Vx,Vy,Vz,eigenvector,omega)

In [17]:
#check
GULP_Vijx=np.loadtxt('Vijx.dat').reshape((natom*3,natom*3))
np.where(np.abs(Vx-GULP_Vijx) > 0.1)

(array([  54,   55,  108,  108,  108,  109,  109,  110,  114,  114,  115,
         116,  117,  117,  117,  118,  118,  119,  132,  132,  133,  133,
         134,  134,  174,  175,  183,  184,  201,  203,  204,  204,  204,
         204,  204,  205,  205,  205,  205,  206,  206,  206,  216,  216,
         217,  218,  225,  225,  225,  225,  226,  226,  226,  227,  315,
         316,  366,  367,  372,  373,  374,  374,  429,  429,  430,  431,
         459,  459,  459,  459,  460,  460,  460,  461,  461,  461,  486,
         487,  488,  488,  510,  511,  579,  579,  580,  581,  624,  625,
         645,  645,  646,  647,  724,  725,  756,  756,  757,  758,  759,
         761,  789,  791,  804,  806,  853,  854,  930,  931,  933,  934,
         957,  958,  959,  959,  960,  961,  999, 1001, 1002, 1002, 1003,
        1004, 1074, 1074, 1075, 1076, 1137, 1139, 1230, 1232, 1266, 1267,
        1440, 1441]),
 array([1294, 1293,  217,  767, 1321,  216, 1320,  765, 1240, 1241, 1239,
        1239,  2

In [25]:
Vx[0,243:260]

array([-0.11037678,  0.05516625,  0.11238344,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ,
       -0.        , -0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

In [26]:
GULP_Vijx[0,243:260]

array([-0.11035713,  0.0387888 ,  0.13676687,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

0,82の要素を詳しくみてみる

In [18]:
masses=atoms.get_masses()

with open(FC_file,'r') as fc:
    lines=fc.readlines()
    nlineblock=4
    fc_all=np.zeros((natom,natom,3,3))
    start=1
    for i in range(natom):
        for j in range(natom):
            fc_block=lines[start+1:start+nlineblock]
            fc=np.loadtxt(fc_block)
            fc_all[i,j]=fc/np.sqrt(masses[i]*masses[j])
            #fc_all[i,j]=fc
            start=start+nlineblock

In [24]:
fc_all[0,81]

array([[-0.10056151,  0.03534579,  0.12462704],
       [ 0.05026058, -0.08582116, -0.14422731],
       [ 0.10238973, -0.09820934, -0.25153559]])

In [38]:
dist[81,0,0]

1.0976046420000003

In [39]:
fc_all[0,81]*dist[81,0,0]

array([[-0.11037678,  0.0387957 ,  0.13679122],
       [ 0.05516625, -0.0941977 , -0.15830457],
       [ 0.11238344, -0.10779503, -0.27608664]])

force constant と距離の積から求めると、GULPの計算とあっている。
tensorのreshapeの挙動がおかしいのかも